# ComCam PTC plots with longer exposure times.

Craig Lage 16-Jul-24

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from lsst.cp.pipe.ptc import PlotPhotonTransferCurveTask, PlotPhotonTransferCurveConfig
%matplotlib inline
from IPython.display import Image, display
from PIL import Image as PILImage
from PIL import ImageDraw as PILImageDraw
import img2pdf

In [ ]:
plotButler = Butler("/repo/embargo", collections=["u/cslage/sdf/comcam/ptc_20240716"])

In [ ]:
filename = "/home/c/cslage/u/ComCam/gains/PTC_20240716.pdf"
plotName = 'ptcVarMean'
imList = []
for detector in range(9):
    ref = plotButler.registry.findDataset(plotName, detector=detector)
    uri = plotButler.getURI(ref)
    imList.append(uri.read())

with open(filename, "wb") as f:
    f.write(img2pdf.convert(imList))
f.close()


In [ ]:
plotNames = ['ptcVarMean', 'ptcVarMeanLog']
for detector in range(9):
    for plotName in plotNames:
        ref = plotButler.registry.findDataset(plotName, detector=detector)
        filename = f"/home/c/cslage/u/ComCam/gains/{plotName}_20240716_{detector}.png"
        uri = plotButler.getURI(ref)
        with open(filename, "wb") as f:
            


In [ ]:
ref = plotButler.registry.findDataset('ptcVarMean', detector=4)

In [ ]:
ref

In [ ]:
uri = plotButler.getURI(ref)
test = display(Image(data=uri.read()))
#plt.savefig("/home/c/cslage/u/ComCam/gains/test.pdf")

In [ ]:
# To save them

detector=0
plotNames = ['ptcVarMean', 'ptcVarMeanLog']
for plotName in plotNames:
    ref = plotButler.registry.findDataset(plotName, detector=detector)
    filename = f"/home/c/cslage/u/AuxTel/gains/{plotName}_20240429_2.png"
    uri = plotButler.getURI(ref)
    with open(filename, "wb") as f:
        f.write(uri.read())


In [ ]:
type(test)

In [ ]:
# Just to view them
plotButler = Butler("/repo/embargo", collections=["u/cslage/sdf/latiss/ptc_20240429_2"])
detector=0
plotNames = ['ptcVarMean', 'ptcVarMeanLog']
for plotName in plotNames:
    ref = plotButler.registry.findDataset(plotName, detector=detector)
    uri = plotButler.getURI(ref)
    display(Image(data=uri.read()))


In [ ]:
# Get the PTC data
ptcButler = Butler("/repo/embargo", collections=["u/cslage/sdf/latiss/ptc_20240429_2"])
ptc = ptcButler.get('ptc', instrument="LATISS", detector=0)


In [ ]:
ptc.rawExpTimes['C10'].shape

In [ ]:
ptc.getExpIdsUsed('C10')

In [ ]:
plt.figure(figsize=(8,8))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle("ComCam Gains - 20240716", fontsize=24)
exposure = 2024071600044
plotcounter = 0
for detector in range(9):
    plotcounter += 1
    plt.subplot(3,3,plotcounter)
    plt.title("Detector%d"%detector, fontsize = 12)
    ptcDataset = plotButler.get('ptc', exposure=exposure, detector=detector)
    gain_data = ptcDataset.gain
    gain_err_data = ptcDataset.gainErr
    amps = gain_data.keys()
    gains = []
    gain_err = []
    amp_nums = []
    for ii, amp in enumerate(amps):
        gains.append(gain_data[amp])
        gain_err.append(gain_err_data[amp])
        amp_nums.append(ii)
    plt.errorbar(amp_nums, gains, yerr=gain_err, marker = 'x')
    plt.ylim(1.4, 1.8)
    plt.ylabel("Gain", fontsize = 12)
    plt.xticks(amp_nums,amps, fontsize=6, rotation=90)
plt.savefig('/home/c/cslage/u/ComCam/gains/Gain_Summary_16Jul24.pdf')
